In [1]:
# Reload automatically when the file is changed.
%load_ext autoreload
%autoreload 2

# Importing Modules

In [2]:
import numpy as np
import torch
import time
import os
from sklearn import (linear_model, model_selection, preprocessing,
                     pipeline)
from scipy.spatial.distance import pdist
from kymatio.torch import HarmonicScattering3D
from kymatio.scattering3d.utils import generate_weighted_sum_of_gaussians
from kymatio.datasets import fetch_qm7
from kymatio.caching import get_cache_dir
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
# Import torch backend 
from kymatio.scattering3d.backend.torch_backend import TorchBackend3D

import os 
import sys
MODULES_PATH = "../../Modules/Scattering"
MODELS_PATH = "../../Models/"

sys.path.append(MODULES_PATH)
sys.path.append(MODELS_PATH)


from Preprocessing import *
from Dataloaders_Preprocessing import *

# Importing Data

In [3]:
List_Data_Train = Load_Data('../../Data/atoms/train', '../../Data/energies/train.csv')
List_Data_Test= Load_Test_Data('../../Data/atoms/test')

In [4]:
List_Data_Train[0]

{'Id': 5415,
 'Atoms_List': [{'Symbol': 'C', 'X': -1.446346, 'Y': -0.436476, 'Z': 0.370742},
  {'Symbol': 'C', 'X': 0.076637, 'Y': -0.504366, 'Z': 0.282528},
  {'Symbol': 'O', 'X': 0.696206, 'Y': -1.438524, 'Z': 0.765824},
  {'Symbol': 'C', 'X': 0.809729, 'Y': 0.603998, 'Z': -0.414085},
  {'Symbol': 'C', 'X': 0.233636, 'Y': 1.6711, 'Z': -0.976108},
  {'Symbol': 'H', 'X': -1.832306, 'Y': -1.305023, 'Z': 0.895733},
  {'Symbol': 'H', 'X': -1.889765, 'Y': -0.404355, 'Z': -0.623347},
  {'Symbol': 'H', 'X': -1.764102, 'Y': 0.460401, 'Z': 0.900525},
  {'Symbol': 'H', 'X': 1.893478, 'Y': 0.499982, 'Z': -0.446412},
  {'Symbol': 'H', 'X': -0.842246, 'Y': 1.81304, 'Z': -0.966871},
  {'Symbol': 'H', 'X': 0.82336, 'Y': 2.438608, 'Z': -1.464688}],
 'Atoms_DataFrame':    Symbol         X         Y         Z
 0       C -1.446346 -0.436476  0.370742
 1       C  0.076637 -0.504366  0.282528
 2       O  0.696206 -1.438524  0.765824
 3       C  0.809729  0.603998 -0.414085
 4       C  0.233636  1.671100 -

In [5]:
Display_Molecule_From_Atom_List(List_Data_Train[500]['Atoms_List'], Width=800, Height=800, Background_Color='lightblue')


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [6]:
Train_Dataset = XYZDataset_V2(List_Data_Train)
Test_Dataset = XYZDataset_V2(List_Data_Test,return_energy=False)

In [7]:
Test_Dataset[0]

(7400,
 tensor([6, 6, 6, 6, 6, 7, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]),
 tensor([4, 4, 4, 4, 4, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]),
 tensor([[ -5.2936,   0.9624, -16.7183],
         [ -8.0905,   5.2601,  -4.8831],
         [ -3.4780,  -2.1574,   4.4672],
         [-10.3931,  -7.1243,  11.9176],
         [  9.1856,  -4.0542,   4.9763],
         [ 15.6663,   6.4253,   3.7869],
         [ -8.8632,   6.5301, -23.0967],
         [ -8.7133,  -7.4531, -18.0707],
         [  3.7848,   0.5602, -18.0829],
         [-17.2584,   5.9623,  -3.9766],
         [ -4.7033,  13.8265,  -3.9112],
         [-19.4660,  -6.0563,  11.4351],
         [ -7.1341, -12.3380,  18.6941],
         [ 11.5159, -10.2823,  -1.6758],
         [ 11.4114,  -8.0106,  13.0612],
         [ 14.1007,  10.1520,  -3.7866],
         [ 24.0789,   4.8126,   4.3437],
         [  0.0000,   0.0000,   0.0000],
         [  0.0000,   0.0000,   0.0000],
         [  0.0000,   0.0000,   0.0000],
         [  0

# Scattering

In [8]:
LOAD_SCATTERING = True

In [9]:
def config(device):
    M = 192
    N = 128
    O = 96
    grid  = torch.tensor(np.fft.ifftshift(np.mgrid[-M//2:-M//2+M, -N//2:-N//2+N, -O//2:-O//2+O]))
    
    J =2
    L = 3
    sigma = 2.0
    integral_powers = [0.5, 1.0, 2.0, 3.0,5.0]
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    scattering = HarmonicScattering3D(J=J, shape=(M, N, O),
                                        L=L, sigma_0=sigma,
                                        integral_powers=integral_powers, max_order=2).to(device)
    
    return grid, scattering, sigma, integral_powers, M,N,O,J,L

In [10]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [11]:
# Define output directory and ensure it exists
output_dir_train = "../../Saved_Features/Scattering/Train"
os.makedirs(output_dir_train, exist_ok=True)

output_dir_test = "../../Saved_Features/Scattering/Test"
os.makedirs(output_dir_test, exist_ok=True)


In [14]:
if not LOAD_SCATTERING:

    order_0, orders_1_and_2 = [], []
    Ids = []
    energies = []
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    grid, scattering, sigma, integral_powers, M,N,O,J,L = config(device)

    test_loader = DataLoader(Test_Dataset, batch_size=16, shuffle=True)
    train_loader = DataLoader(Train_Dataset, batch_size=16, shuffle=True)


    import torch
    from tqdm.auto import tqdm
    import os

    # Assuming order_0, orders_1_and_2, and energies are lists
    order_0 = []
    orders_1_and_2 = []
    energies = []

    # Your existing loop
    for batch in tqdm(train_loader):
        Id_Batch = batch[0]
        full_charge_batch = batch[1]
        valence_batch = batch[2]
        pos_batch = batch[3]
        energie = batch[4]

        full_density_batch = generate_weighted_sum_of_gaussians(grid, pos_batch, full_charge_batch, sigma)
        full_density_batch = torch.from_numpy(full_density_batch)
        full_density_batch = full_density_batch.to(device).float()
        full_order_0 = TorchBackend3D.compute_integrals(full_density_batch, integral_powers)
        full_scattering = scattering(full_density_batch)

        val_density_batch = generate_weighted_sum_of_gaussians(grid, pos_batch, valence_batch, sigma)
        val_density_batch = torch.from_numpy(val_density_batch)
        val_density_batch = val_density_batch.to(device).float()
        val_order_0 = TorchBackend3D.compute_integrals(val_density_batch, integral_powers)
        val_scattering = scattering(val_density_batch)

        core_density_batch = full_density_batch - val_density_batch
        core_order_0 = TorchBackend3D.compute_integrals(core_density_batch, integral_powers)
        core_scattering = scattering(core_density_batch)

        batch_order_0 = torch.stack((full_order_0, val_order_0, core_order_0), dim=-1)
        batch_orders_1_and_2 = torch.stack((full_scattering, val_scattering, core_scattering), dim=-1)
        
        order_0.append(batch_order_0)
        orders_1_and_2.append(batch_orders_1_and_2)
        energies.append(energie)
        Ids.append(Id_Batch)
else:
    order_0_train = torch.load(os.path.join(output_dir_train, 'order_0.pt'))
    orders_1_and_2_train = torch.load(os.path.join(output_dir_train, 'orders_1_and_2.pt'))
    energies_train = torch.load(os.path.join(output_dir_train, 'energies.pt'))
    Ids_train = torch.load(os.path.join(output_dir_train, 'Ids.pt'))

    


/tmp/ipykernel_426879/2641103653.py:55: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  order_0_train = torch.load(os.path.join(output_dir_train, 'order_0.pt'))
/tmp/ipykernel

In [ ]:
if not LOAD_SCATTERING:
    # After the loop, concatenate the lists into single tensors
    order_0_train = torch.cat(order_0, dim=0)  # Adjust dim as needed
    orders_1_and_2_train = torch.cat(orders_1_and_2, dim=0)  # Adjust dim as needed
    energies_train = torch.cat(energies, dim=0)  # Adjust dim as needed
    Ids_train = torch.cat(Ids, dim=0)  # Adjust dim as needed

    # Save tensors to files
    torch.save(order_0_train, os.path.join(output_dir_train, "order_0.pt"))
    torch.save(orders_1_and_2_train, os.path.join(output_dir_train, "orders_1_and_2.pt"))
    torch.save(energies_train, os.path.join(output_dir_train, "energies.pt"))

    # Save Ids
    torch.save(Ids_train, os.path.join(output_dir_train, "Ids.pt"))



In [16]:
if not LOAD_SCATTERING:
    order_0, orders_1_and_2 = [], []
    energies = []
    Ids = []
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    grid, scattering, sigma, integral_powers, M,N,O,J,L = config(device)

    test_loader = DataLoader(Test_Dataset, batch_size=16, shuffle=False)


    import torch
    from tqdm.auto import tqdm
    import os

    # Assuming order_0, orders_1_and_2, and energies are lists
    order_0 = []
    orders_1_and_2 = []
    energies = []

    # Your existing loop
    for batch in tqdm(test_loader):
        Id_batch = batch[0]
        full_charge_batch = batch[1]
        valence_batch = batch[2]
        pos_batch = batch[3]
        energie = 0

        full_density_batch = generate_weighted_sum_of_gaussians(grid, pos_batch, full_charge_batch, sigma)
        full_density_batch = torch.from_numpy(full_density_batch)
        full_density_batch = full_density_batch.to(device).float()
        full_order_0 = TorchBackend3D.compute_integrals(full_density_batch, integral_powers)
        full_scattering = scattering(full_density_batch)

        val_density_batch = generate_weighted_sum_of_gaussians(grid, pos_batch, valence_batch, sigma)
        val_density_batch = torch.from_numpy(val_density_batch)
        val_density_batch = val_density_batch.to(device).float()
        val_order_0 = TorchBackend3D.compute_integrals(val_density_batch, integral_powers)
        val_scattering = scattering(val_density_batch)

        core_density_batch = full_density_batch - val_density_batch
        core_order_0 = TorchBackend3D.compute_integrals(core_density_batch, integral_powers)
        core_scattering = scattering(core_density_batch)

        batch_order_0 = torch.stack((full_order_0, val_order_0, core_order_0), dim=-1)
        batch_orders_1_and_2 = torch.stack((full_scattering, val_scattering, core_scattering), dim=-1)
        
        order_0.append(batch_order_0)
        orders_1_and_2.append(batch_orders_1_and_2)
        energies.append(energie)
        Ids.append(Id_batch)
else:
    order_0_test = torch.load(os.path.join(output_dir_test, 'order_0.pt'))
    orders_1_and_2_test = torch.load(os.path.join(output_dir_test, 'orders_1_and_2.pt'))
    # energies_test = torch.load(os.path.join(output_dir_test, 'energies.pt'))
    Ids_test = torch.load(os.path.join(output_dir_test, 'Ids.pt'))

    



/tmp/ipykernel_426879/979074118.py:53: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  order_0_test = torch.load(os.path.join(output_dir_test, 'order_0.pt'))
/tmp/ipykernel_42

In [17]:
if not LOAD_SCATTERING:
    # After the loop, concatenate the lists into single tensors
    order_0_test = torch.cat(order_0, dim=0)  # Adjust dim as needed
    orders_1_and_2_test = torch.cat(orders_1_and_2, dim=0)  # Adjust dim as needed
    # energies_tensor = torch.cat(energies, dim=0)  # Adjust dim as needed
    Ids_tensor = torch.cat(Ids, dim=0)  # Adjust dim as needed
    torch.save(order_0_test, os.path.join(output_dir_test, "order_0.pt"))
    torch.save(orders_1_and_2_test, os.path.join(output_dir_test, "orders_1_and_2.pt"))
    torch.save(Ids_tensor, os.path.join(output_dir_test, "Ids.pt"))


# XGBoost

In [18]:
import torch
import numpy as np
import xgboost as xgb
from torch import nn
from sklearn.metrics import mean_squared_error, accuracy_score
import logging

# Configuration du logger
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class XGBoostWrapper(nn.Module):
    def __init__(self, input_dim, n_estimators=100, max_depth=6, learning_rate=0.1,
                objective='reg:squarederror', model_type='regressor',
                reg_lambda=1.0, reg_alpha=0.0):
        super().__init__()
        self.input_dim = input_dim
        self.model_type = model_type.lower()
        self.objective = objective
        self.params = {
            'max_depth': max_depth,
            'eta': learning_rate,
            'objective': objective,
            'eval_metric': 'rmse' if self.model_type == 'regressor' else 'mlogloss',
            'tree_method': 'hist',
            'verbosity': 0,
            'seed': 42,
            'lambda': reg_lambda,   # régularisation L2
            'alpha': reg_alpha      # régularisation L1
        }
        self.n_estimators = n_estimators
        self.booster = None
        self.is_fitted = False
        logger.info(f"XGBoostWrapper initialisé avec input_dim={input_dim}, model_type={model_type}, reg_lambda={reg_lambda}, reg_alpha={reg_alpha}")
    def fit(self, X, y, eval_set=None, early_stopping_rounds=None, verbose=False):
        if isinstance(X, torch.Tensor):
            X = X.cpu().numpy()
        if isinstance(y, torch.Tensor):
            y = y.cpu().numpy().ravel()

        dtrain = xgb.DMatrix(X, label=y)

        evals = [(dtrain, 'train')]
        if eval_set is not None:
            eval_X, eval_y = eval_set[0]
            if isinstance(eval_X, torch.Tensor):
                eval_X = eval_X.cpu().numpy()
            if isinstance(eval_y, torch.Tensor):
                eval_y = eval_y.cpu().numpy().ravel()
            dval = xgb.DMatrix(eval_X, label=eval_y)
            evals.append((dval, 'eval'))

        try:
            logger.info(f"Early stopping activé avec {early_stopping_rounds} rounds")
            self.booster = xgb.train(
                params=self.params,
                dtrain=dtrain,
                num_boost_round=self.n_estimators,
                evals=evals,
                early_stopping_rounds=early_stopping_rounds,
                verbose_eval=verbose
            )
            self.is_fitted = True
            logger.info("Entraînement terminé avec succès")
        except Exception as e:
            logger.error(f"Erreur dans fit() : {e}")
            raise e

    def forward(self, x):
        if not self.is_fitted:
            raise RuntimeError("Le modèle doit être entraîné avant d'utiliser forward().")
        if isinstance(x, torch.Tensor):
            x_np = x.cpu().numpy()
        else:
            x_np = x
        dmatrix = xgb.DMatrix(x_np)
        preds = self.booster.predict(dmatrix)
        preds = torch.tensor(preds, dtype=torch.float32)
        if preds.dim() == 1:
            preds = preds.unsqueeze(-1)
        return preds

    def evaluate(self, X, y=None):
        if not self.is_fitted:
            raise RuntimeError("Le modèle doit être entraîné avant d'évaluer.")
        if isinstance(X, torch.Tensor):
            X = X.cpu().numpy()
        dmatrix = xgb.DMatrix(X)
        preds = self.booster.predict(dmatrix)
        preds_list = preds.tolist()

        metric = None
        if y is not None:
            if isinstance(y, torch.Tensor):
                y = y.cpu().numpy().ravel()
            if self.model_type == 'regressor':
                metric = mean_squared_error(y, preds)
                logger.info(f"MSE : {metric:.4f}")
            elif self.model_type == 'classifier':
                pred_labels = (preds > 0.5).astype(int)
                metric = accuracy_score(y, pred_labels)
                logger.info(f"Accuracy : {metric:.4f}")
        return preds_list, metric

    def save_model(self, path):
        if not self.is_fitted:
            raise RuntimeError("Le modèle n'est pas encore entraîné.")
        self.booster.save_model(path)
        logger.info(f"Modèle sauvegardé : {path}")

    def load_model(self, path):
        self.booster = xgb.Booster()
        self.booster.load_model(path)
        self.is_fitted = True
        logger.info(f"Modèle chargé : {path}")

    def get_params(self):
        return self.params

In [26]:
import os
import torch
import numpy as np
from sklearn.linear_model import RidgeCV
from sklearn.metrics import mean_squared_error

# -----------------------------
# 1️⃣ Charger les tensors
# -----------------------------
output_dir_train = "../../Saved_Features/Scattering/Train"

order_0_train = torch.load(os.path.join(output_dir_train, "order_0.pt"))
orders_1_and_2_train = torch.load(os.path.join(output_dir_train, "orders_1_and_2.pt"))
energies_train = torch.load(os.path.join(output_dir_train, "energies.pt"))
Ids_train = torch.load(os.path.join(output_dir_train, "Ids.pt"))

# -----------------------------
# 2️⃣ Mise sur CPU et flatten
# -----------------------------
order_0_train = order_0_train.cpu().reshape(order_0_train.shape[0], -1)
orders_1_and_2_train = orders_1_and_2_train.cpu().reshape(orders_1_and_2_train.shape[0], -1)
energies_train = energies_train.cpu()
Ids_train = Ids_train.cpu()

# -----------------------------
# 3️⃣ Conversion en NumPy
# -----------------------------
order_0_train_np = order_0_train.numpy()
orders_1_and_2_train_np = orders_1_and_2_train.numpy()
energies_train_np = energies_train.numpy()
Ids_train_np = Ids_train.numpy()




print("Shape of order_0_train_np:", order_0_train_np.shape)
print("Shape of orders_1_and_2_train_np:", orders_1_and_2_train_np.shape)
print("Shape of energies_train_np:", energies_train_np.shape)

# -----------------------------
# 4️⃣ Combiner en X_train
# -----------------------------
X_train = np.hstack((order_0_train_np, orders_1_and_2_train_np))


# -------------------------------
# Céation de données de Val
# -----------------------

X_train , X_val, energies_train_np, energies_val_np = model_selection.train_test_split(
    X_train, energies_train_np, test_size=0.2, random_state=42
)

# -----------------------------
# 5️⃣ Entraîner le modèle XGBoost
# -----------------------------

xgb_model = XGBoostWrapper(input_dim=X_train.shape[1], n_estimators=10000, max_depth=8, learning_rate=0.01,
                          objective='reg:squarederror', model_type='regressor',
                          reg_lambda=1.0, reg_alpha=0.0)
xgb_model.fit(X_train, energies_train_np, early_stopping_rounds=10,
                eval_set=[(X_val, energies_val_np)], verbose=True)


/tmp/ipykernel_426879/924070461.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  order_0_train = torch.load(os.path.join(output_dir_train, "order_0.pt"))
/tmp/ipykernel_

Shape of order_0_train_np: (6591, 15)
Shape of orders_1_and_2_train_np: (6591, 360)
Shape of energies_train_np: (6591,)
[0]	train-rmse:11.52846	eval-rmse:11.57582
[1]	train-rmse:11.41501	eval-rmse:11.46130
[2]	train-rmse:11.30269	eval-rmse:11.34798
[3]	train-rmse:11.19149	eval-rmse:11.23574
[4]	train-rmse:11.08140	eval-rmse:11.12464
[5]	train-rmse:10.97241	eval-rmse:11.01465
[6]	train-rmse:10.86451	eval-rmse:10.90556
[7]	train-rmse:10.75767	eval-rmse:10.79778
[8]	train-rmse:10.65191	eval-rmse:10.69082
[9]	train-rmse:10.54719	eval-rmse:10.58521
[10]	train-rmse:10.44351	eval-rmse:10.48059
[11]	train-rmse:10.34087	eval-rmse:10.37700
[12]	train-rmse:10.23922	eval-rmse:10.27506
[13]	train-rmse:10.13859	eval-rmse:10.17346
[14]	train-rmse:10.03898	eval-rmse:10.07295
[15]	train-rmse:9.94035	eval-rmse:9.97337
[16]	train-rmse:9.84267	eval-rmse:9.87528
[17]	train-rmse:9.74601	eval-rmse:9.77776
[18]	train-rmse:9.65029	eval-rmse:9.68112
[19]	train-rmse:9.55554	eval-rmse:9.58561
[20]	train-rmse:9.46

2025-06-20 08:34:46,094 - INFO - Entraînement terminé avec succès


In [30]:
# predict on test set
output_dir_test = "../../Saved_Features/Scattering/Test"
order_0_test = torch.load(os.path.join(output_dir_test, "order_0.pt"))
orders_1_and_2_test = torch.load(os.path.join(output_dir_test, "orders_1_and_2.pt"))
Ids_test = torch.load(os.path.join(output_dir_test, "Ids.pt"))

# -----------------------------
# 1️⃣ Mise sur CPU et flatten
# -----------------------------
order_0_test = order_0_test.cpu().reshape(order_0_test.shape[0], -1)
orders_1_and_2_test = orders_1_and_2_test.cpu().reshape(orders_1_and_2_test.shape[0], -1)
# -----------------------------
# 2️⃣ Conversion en NumPy
# -----------------------------
order_0_test_np = order_0_test.numpy()
orders_1_and_2_test_np = orders_1_and_2_test.numpy()        
# -----------------------------
# 3️⃣ Combiner en X_test
# -----------------------------
X_test = np.hstack((order_0_test_np, orders_1_and_2_test_np))

# -----------------------------

# 4️⃣ Prédiction sur le test set
# -----------------------------
energies_test_pred = xgb_model.forward(X_test)
# Afficher les résultats
for id, energy in zip(Ids_test, energies_test_pred):
    print(f"ID: {id.item()}, Predicted Energy: {energy.item():.4f}")
# Save predictions as a CSV file
import pandas as pd
predictions_df = pd.DataFrame({
    'id': Ids_test.numpy(),
    'energy': energies_test_pred.cpu().numpy().ravel()
})

predictions_df.to_csv('../../Saved_Features/Scattering/predictions_xgboost.csv', index=False)


ID: 7400, Predicted Energy: -76.3627
ID: 7691, Predicted Energy: -67.1279
ID: 7584, Predicted Energy: -72.8368
ID: 7356, Predicted Energy: -74.9270
ID: 8122, Predicted Energy: -89.8651
ID: 7594, Predicted Energy: -97.8483
ID: 7654, Predicted Energy: -77.9414
ID: 7972, Predicted Energy: -103.1919
ID: 7069, Predicted Energy: -83.4677
ID: 7149, Predicted Energy: -90.0757
ID: 7656, Predicted Energy: -74.5799
ID: 7123, Predicted Energy: -84.5011
ID: 7762, Predicted Energy: -84.1583
ID: 6752, Predicted Energy: -64.8419
ID: 7212, Predicted Energy: -76.3481
ID: 7830, Predicted Energy: -74.8304
ID: 8154, Predicted Energy: -83.1078
ID: 7155, Predicted Energy: -69.7283
ID: 7042, Predicted Energy: -91.1072
ID: 7007, Predicted Energy: -69.1479
ID: 7625, Predicted Energy: -83.0649
ID: 7898, Predicted Energy: -97.0690
ID: 8171, Predicted Energy: -74.6981
ID: 7627, Predicted Energy: -90.2123
ID: 6853, Predicted Energy: -76.0671
ID: 6979, Predicted Energy: -85.0120
ID: 7120, Predicted Energy: -71.4691


/tmp/ipykernel_426879/1770267278.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  order_0_test = torch.load(os.path.join(output_dir_test, "order_0.pt"))
/tmp/ipykernel_42